In [1]:
from pyspark.sql import SparkSession
import os
import json
from datetime import timedelta,datetime
from pyspark.sql.functions import *
import time

In [2]:
spark = SparkSession \
    .builder \
    .getOrCreate()

22/09/17 18:05:03 WARN Utils: Your hostname, DESKTOP-GPLR42E resolves to a loopback address: 127.0.1.1; using 172.25.231.107 instead (on interface eth0)
22/09/17 18:05:03 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/09/17 18:05:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
base_path = '/mnt/data/json'
override_config_key = 'spark.override.date'
parquet_write_path = '/mnt/data/parquet/'
current_date = datetime.now().strftime("%Y-%m-%d")

In [4]:
try: 
    conf_value = spark.conf.get(override_config_key) 
    paths_to_read = '{}/date={}'.format(base_path,conf_value)
except:
    print("Parameter is not set")
    paths_to_read = '{}/date={}'.format(base_path,current_date)


Parameter is not set


In [5]:
df = spark.read.option("basePath",base_path) \
    .option("multiLine", "True") \
        .json(paths_to_read)

In [6]:
final_dataframe = df.withColumn("source_name",col("source.name")) \
    .withColumn("timestamp",to_timestamp("publishedAt")) \
    .drop("id","title","urlToImage","source","publishedAt")

In [7]:
final_dataframe.write.mode("overwrite") \
    .partitionBy("date") \
    .option("partitionOverwriteMode","dynamic") \
    .parquet(parquet_write_path)